# Notebook 5: Clean Up Resources

Specify "Python 3" Kernel  and "Data Science" Image.

### Background

In this notebook, we will clean up the resources we provisioned during this workshop:

- SageMaker Feature Groups
- SageMaker Endpoints
- Amazon Kinesis Data Stream
- Amazon Kinesis Data Analytics application

### Imports

In [29]:
from parameter_store import ParameterStore
from utils import *

### Session variables

In [30]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
boto_session = boto3.Session()
kinesis_client = boto_session.client(service_name='kinesis', region_name=region)
kinesis_analytics_client = boto_session.client('kinesisanalytics')

ps = ParameterStore(verbose=False)
ps.set_namespace('feature-store-workshop')

Load variables from previous notebooks

In [31]:
parameters = ps.read()

customers_feature_group_name = parameters['customers_feature_group_name']
products_feature_group_name = parameters['products_feature_group_name']
orders_feature_group_name = parameters['orders_feature_group_name']
click_stream_historical_feature_group_name = parameters['click_stream_historical_feature_group_name']
click_stream_feature_group_name = parameters['click_stream_feature_group_name']

cf_model_endpoint_name = parameters['cf_model_endpoint_name']
ranking_model_endpoint_name = parameters['ranking_model_endpoint_name']

kinesis_stream_name = parameters['kinesis_stream_name']
kinesis_analytics_application_name = parameters['kinesis_analytics_application_name']

### Delete feature groups

In [32]:
feature_group_list = [customers_feature_group_name, products_feature_group_name,
                      orders_feature_group_name, click_stream_historical_feature_group_name,
                      click_stream_feature_group_name]

for feature_group in feature_group_list:
    print(f'Deleting feature group: {feature_group}')
    delete_feature_group(feature_group)

Deleting feature group: recsys-customers-fg-05-28-23-12
Deleting all s3 objects in prefix: recsys-feature-store/XXXXXXXXXXXX/sagemaker/ap-southeast-2/offline-store/recsys-customers-fg-05-28-23-12 in bucket sagemaker-ap-southeast-2-XXXXXXXXXXXX
Waiting for Feature Group Deletion
Deleting feature group: recsys-products-fg-05-28-23-12
Deleting all s3 objects in prefix: recsys-feature-store/XXXXXXXXXXXX/sagemaker/ap-southeast-2/offline-store/recsys-products-fg-05-28-23-12 in bucket sagemaker-ap-southeast-2-XXXXXXXXXXXX
Waiting for Feature Group Deletion
Deleting feature group: recsys-orders-fg-05-28-23-12
Deleting all s3 objects in prefix: recsys-feature-store/XXXXXXXXXXXX/sagemaker/ap-southeast-2/offline-store/recsys-orders-fg-05-28-23-12 in bucket sagemaker-ap-southeast-2-XXXXXXXXXXXX
Waiting for Feature Group Deletion
Deleting feature group: recsys-click-stream-historical-fg-05-28-23-12
Deleting all s3 objects in prefix: recsys-feature-store/XXXXXXXXXXXX/sagemaker/ap-southeast-2/offline

### Delete endpoints and endpoint configurations

In [33]:
def clean_up_endpoint(endpoint_name):
    response = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_config_name = response['EndpointConfigName']
    print(f'Deleting endpoint: {endpoint_name}')
    print(f'Deleting endpoint configuration : {endpoint_config_name}')
    sagemaker_session.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    sagemaker_session.sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

In [34]:
endpoint_list = [cf_model_endpoint_name, ranking_model_endpoint_name]

for endpoint in endpoint_list:
    clean_up_endpoint(endpoint)

Deleting endpoint: recsys-cf-model-05-28-23-35
Deleting endpoint configuration : recsys-cf-model-05-28-23-35
Deleting endpoint: recsys-rerank-model-05-28-23-35
Deleting endpoint configuration : recsys-rerank-model-05-28-23-35


### Delete Kinesis Data Stream

In [35]:
kinesis_client.delete_stream(StreamName=kinesis_stream_name,
                             EnforceConsumerDeletion=True)

{'ResponseMetadata': {'RequestId': 'f78be708-b4a7-d2de-a95b-934960b27d21',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f78be708-b4a7-d2de-a95b-934960b27d21',
   'x-amz-id-2': 'XP2LB+4JbsvKcU5fzgjjolKNwu/VscIteAKJME7nnFnYkN/XL9mdTJo8RFXmZu0APzfUXz7F0myVLYKFb1SWXj8qrvucOao8',
   'date': 'Wed, 29 May 2024 00:08:33 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Delete Kinesis Data Analytics application

In [36]:
response = kinesis_analytics_client.describe_application(ApplicationName=kinesis_analytics_application_name)
create_ts = response['ApplicationDetail']['CreateTimestamp']
kinesis_analytics_client.delete_application(ApplicationName=kinesis_analytics_application_name, CreateTimestamp=create_ts)

{'ResponseMetadata': {'RequestId': '334ef27e-7207-485b-bb2b-8b97a99da0f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '334ef27e-7207-485b-bb2b-8b97a99da0f8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Wed, 29 May 2024 00:08:33 GMT'},
  'RetryAttempts': 0}}

Go back to Workshop Studio and click on "Next".